In [1]:
import os
os.chdir('../src')

from utils import data_utils

import torch
import torch.nn as nn
import torch.optim as optim

import pandas as pd
import numpy as np
import anndata as ad
import scanpy as sc
pd.set_option("display.max_columns", None)

import torch; torch.manual_seed(100)
import torch.nn as nn
import torch.utils
import torch.distributions
import torchvision
from torchvision import datasets, transforms
import math
import numpy as np
np.random.seed(100)
import matplotlib.pyplot as plt; plt.rcParams['figure.dpi'] = 200
import scanpy as sc
from collections import OrderedDict
from collections import Counter

# select the right device, depending on whether your Colab runs on GPU or CPU
### IMPORTANT: we recommend to change your runtime to GPU, otherwise the training takes much longer
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


TODO


In [2]:
adata = data_utils.load_anndata(mode='train', plus_iid_holdout=False, preprocessing='log1p')
# sc.pp.scale(data)

d:\dlls-hack\.venv\Lib\site-packages\anndata\_core\anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


Loading precomputed log1p...


d:\dlls-hack\.venv\Lib\site-packages\anndata\_core\anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [3]:
adata.obs.columns

Index(['GEX_n_genes_by_counts', 'GEX_pct_counts_mt', 'GEX_size_factors',
       'GEX_phase', 'ADT_n_antibodies_by_counts', 'ADT_total_counts',
       'ADT_iso_count', 'cell_type', 'batch', 'ADT_pseudotime_order',
       'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality',
       'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType',
       'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker',
       'is_train'],
      dtype='object')

In [4]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.pca(adata, n_comps=50)
sc.external.pp.bbknn(adata, batch_key='Site')

In [7]:
sc.tl.umap(adata)
# sc.tl.leiden(adata)

In [9]:
adata.X.toarray().shape

(66175, 14087)

In [27]:
adata.obsm['X_pca'].shape

(66175, 50)

In [11]:
!pip install harmonypy

In [12]:
sc.external.pp.harmony_integrate(data, key='batch')

KeyError: 'X_pca'

In [39]:
BATCH_SIZE = 128

In [40]:
train_dataloader = data_utils.get_dataloader_from_anndata(data[:-1000, :], first_modality_dim=10, second_modality_dim=134, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = data_utils.get_dataloader_from_anndata(data[-1000:, :], first_modality_dim=10, second_modality_dim=134, batch_size=BATCH_SIZE, shuffle=True)
del data

In [41]:
class Encoder(nn.Module):
    def __init__(self, latent_dims, input_dims, dropout, z_dropout): #dropout, z_dropout define the dropout rates of the encoder/latent space
        super(Encoder, self).__init__()
        self.encoder = nn.Sequential(nn.Linear(input_dims, 800),
                                     nn.ReLU(),
                                     nn.Dropout(p = dropout),
                                     nn.Linear(800, 800),
                                     nn.ReLU(),
                                     nn.Dropout(p = dropout))  #two layer, fully connected encoder with dropout


        self.mu = nn.Sequential(nn.Linear(800, latent_dims),
                                nn.Dropout(p = z_dropout))

        self.sigma = nn.Sequential(nn.Linear(800, latent_dims),
                                   nn.Dropout(p = z_dropout))

        self.N = torch.distributions.Normal(0, 1)
        self.N.loc = self.N.loc.to(device)
        self.N.scale = self.N.scale.to(device)
        self.kl = 0

    def forward(self, x):
        x = self.encoder(x)
        mu =  self.mu(x)
        sigma = torch.exp(self.mu(x)) # exp for numeric stability
        z = mu + sigma*self.N.sample(mu.shape)
        self.kl = (0.5*sigma**2 + 0.5*mu**2 - torch.log(sigma) - 1/2).sum() #calculation of kullback-leibler divergence

        return z

class Decoder(nn.Module):
    def __init__(self, latent_dims, input_dims, dropout):
        super(Decoder, self).__init__()
        self.decoder = nn.Sequential(nn.Linear(latent_dims, 800),
                                     nn.ReLU(),
                                     nn.Dropout(p = dropout),
                                     nn.Linear(800, input_dims)
                                     )

    def forward(self, z):
        z = self.decoder(z)
        return z

class VariationalAutoencoder(nn.Module):
    def __init__(self, latent_dims, input_dims,dropout = 0.3, z_dropout=0.3):
        super(VariationalAutoencoder, self).__init__()
        self.encoder = Encoder(latent_dims, input_dims, dropout, z_dropout)
        self.decoder = Decoder(latent_dims, input_dims, dropout)

    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)

def train(vae, data, epochs=50, beta = 0.001, learning_rate = 0.0001):
    opt = torch.optim.Adam(vae.parameters(), lr = learning_rate) # ADAM optimizer
    losses = []
    klds = []
    mses = []
    vae.train() #train mode (then, dropout layers are considered)

    for epoch in range(epochs):
        loss_e = 0
        kld_e = 0
        mse_e = 0

        for _, x in data: # we only care about the second modality
            x = x.to(device)
            opt.zero_grad()
            x_hat = vae(x)
            mse = ((x - x_hat)**2).sum()
            kld = beta* vae.encoder.kl
            loss = mse +  kld # loss calculation
            loss.backward()
            opt.step()
            loss_e += loss.to('cpu').detach().numpy()
            kld_e += kld.to('cpu').detach().numpy()
            mse_e += mse.to('cpu').detach().numpy()

        losses.append(loss_e/(len(data)*BATCH_SIZE))
        klds.append(kld_e/(len(data)*BATCH_SIZE))
        mses.append(mse_e/(len(data)*BATCH_SIZE))

        print("epoch: ", epoch, " loss: ", loss_e/(len(data)*BATCH_SIZE))

    return vae, losses, klds, mses

In [42]:
latent_dims = 5 #choose the number of latent variables

In [43]:
next(iter(train_dataloader))[1].shape

torch.Size([128, 134])

In [48]:
next(iter(train_dataloader))[1]

tensor([[  1.,   2.,   3.,  ...,  34.,   5.,  13.],
        [  1.,   1.,   7.,  ...,   1.,   5.,   4.],
        [  2.,   9.,  32.,  ...,  31.,  17.,   7.],
        ...,
        [  1.,   2.,   9.,  ..., 151.,   5.,   7.],
        [  0.,   1.,  19.,  ..., 143.,   3.,  27.],
        [  4.,   0.,   2.,  ...,  13.,   7.,   9.]])

In [44]:
vae = VariationalAutoencoder(latent_dims = latent_dims, input_dims = 134).to(device)
vae, losses, klds, mses = train(vae, train_dataloader, beta = 0.001) # takes about 1 min on GPU

epoch:  0  loss:  nan
epoch:  1  loss:  nan
epoch:  2  loss:  nan
epoch:  3  loss:  nan
epoch:  4  loss:  nan
epoch:  5  loss:  nan
epoch:  6  loss:  nan
epoch:  7  loss:  nan
epoch:  8  loss:  nan
epoch:  9  loss:  nan
epoch:  10  loss:  nan
epoch:  11  loss:  nan
epoch:  12  loss:  nan
epoch:  13  loss:  nan
epoch:  14  loss:  nan


KeyboardInterrupt: 

In [ ]:
data.obs

In [51]:
scaled_data = sc.pp.scale(data)

d:\dlls-hack\.venv\Lib\site-packages\scanpy\preprocessing\_scale.py:299: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
d:\dlls-hack\.venv\Lib\site-packages\anndata\_core\anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


MemoryError: Unable to allocate 3.47 GiB for an array with shape (66175, 14087) and data type float32

In [ ]:
scaled_data.layers

In [47]:
sc.external.pp.bbknn(data, batch_key='batch')

ImportError: Please install bbknn: `pip install bbknn`.